In [9]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [10]:
pickle_file='notMNIST.pickle'

with open(pickle_file,'rb') as f:
    save=pickle.load(f)
    train_dataset=save['train_dataset']
    train_labels=save['train_labels']
    valid_dataset=save['valid_dataset']
    valid_labels=save['valid_labels']
    test_dataset=save['test_dataset']
    test_labels=save['test_labels']
    del save
    print('Training dataset',train_dataset.shape,train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Testing dataset',test_dataset.shape,test_labels.shape)


Training dataset (121885, 28, 28) (121885,)
Validation set (10000, 28, 28) (10000,)
Testing dataset (4620, 28, 28) (4620,)


In [11]:
image_size=28
num_labels=10

def reformat(dataset,labels):
    dataset=dataset.reshape(-1,image_size*image_size).astype(np.float32)
    labels=(np.arange(num_labels)==labels[:,None]).astype(np.float32)
    return dataset , labels
train_dataset,train_labels=reformat(train_dataset,train_labels)
test_dataset,test_labels=reformat(test_dataset,test_labels)
valid_dataset,valid_labels=reformat(valid_dataset,valid_labels)
print('Training dataset',train_dataset.shape,train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Testing dataset',test_dataset.shape,test_labels.shape)    

Training dataset (121885, 784) (121885, 10)
Validation set (10000, 784) (10000, 10)
Testing dataset (4620, 784) (4620, 10)


In [12]:
#tensorflow part
train_subset=10000

graph=tf.Graph()
with graph.as_default():
    
    #input read
    tf_train_dataset=tf.constant(train_dataset[:train_subset,:])
    tf_train_labels=tf.constant(train_labels[:train_subset,:])
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    #variables
    weights=tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases=tf.Variable(tf.zeros([num_labels]))
    
    #training computation
    logits=tf.matmul(tf_train_dataset,weights)+biases
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
    
    #optimizer
    optimizer=tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_predictions=tf.nn.softmax(logits)
    test_predicitions=tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+biases)
    valid_predicitons=tf.nn.softmax(tf.matmul(tf_valid_dataset,weights)+biases)
    
    
    


In [13]:
num_steps=801


def accuracy(predictions,labels):
      return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range(num_steps):
        _,l,predicitions=session.run([optimizer,loss,train_predictions])
        if(step%100==0):
            print('Loss at step %d: %f'%(step,l))
            print('Training accuracy: %.1f%%'%accuracy(predicitions,train_labels[:train_subset,:]))
            
            print('Validation accuracy %.1f%%'%accuracy(valid_predicitons.eval(),valid_labels))
            
    print('Test accuracy: %.1f%%'%accuracy(test_predicitions.eval(),test_labels))
              
          
          
        
          

initialized
Loss at step 0: 20.055002
Training accuracy: 7.3%
Validation accuracy 8.4%
Loss at step 100: 2.296377
Training accuracy: 71.8%
Validation accuracy 70.6%
Loss at step 200: 1.816701
Training accuracy: 75.4%
Validation accuracy 73.6%
Loss at step 300: 1.575947
Training accuracy: 76.7%
Validation accuracy 74.5%
Loss at step 400: 1.416318
Training accuracy: 77.5%
Validation accuracy 75.0%
Loss at step 500: 1.299955
Training accuracy: 78.4%
Validation accuracy 75.3%
Loss at step 600: 1.209390
Training accuracy: 78.9%
Validation accuracy 75.6%
Loss at step 700: 1.135837
Training accuracy: 79.4%
Validation accuracy 75.8%
Loss at step 800: 1.074277
Training accuracy: 79.9%
Validation accuracy 75.7%
Test accuracy: 83.3%


In [14]:
#stochastoc gradient descent

batch_size=128

graph=tf.Graph()

with graph.as_default():
    
    #input
    tf_train_dataset=tf.placeholder(np.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


In [15]:
#run

num_steps=3001
 
with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('initialized')
    
    for step in range(num_steps):
        
        offset=(step*batch_size)%(train_labels.shape[0]-batch_size)
        
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
        

initialized
Minibatch loss at step 0: 13.985790
Minibatch accuracy: 10.9%
Validation accuracy: 13.0%
Minibatch loss at step 500: 2.203452
Minibatch accuracy: 67.2%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 1.188179
Minibatch accuracy: 77.3%
Validation accuracy: 77.0%
Minibatch loss at step 1500: 1.359336
Minibatch accuracy: 73.4%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 0.922862
Minibatch accuracy: 76.6%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 0.989969
Minibatch accuracy: 78.1%
Validation accuracy: 78.4%
Minibatch loss at step 3000: 0.881056
Minibatch accuracy: 79.7%
Validation accuracy: 79.0%
Test accuracy: 85.5%


In [25]:
#Problem

hidden_size=1024

graph=tf.Graph()

with graph.as_default():
    
    tf_train_dataset=tf.placeholder(np.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    
    hidden_1=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    
    weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    logits=tf.matmul(hidden_1,weights_2)+biases_2
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights_1) + biases_1)),weights_2)+biases_2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_1) + biases_1)),weights_2)+biases_2)

In [26]:

num_steps=3001
 
with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    print('initialized')
    
    for step in range(num_steps):
        
        offset=(step*batch_size)%(train_labels.shape[0]-batch_size)
        
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

initialized
Minibatch loss at step 0: 377.989838
Minibatch accuracy: 10.9%
Validation accuracy: 29.2%
Minibatch loss at step 500: 19.684687
Minibatch accuracy: 78.1%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 9.062372
Minibatch accuracy: 82.8%
Validation accuracy: 82.7%
Minibatch loss at step 1500: 11.048413
Minibatch accuracy: 85.2%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 6.033473
Minibatch accuracy: 85.9%
Validation accuracy: 83.6%
Minibatch loss at step 2500: 15.972140
Minibatch accuracy: 76.6%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 8.238071
Minibatch accuracy: 83.6%
Validation accuracy: 82.7%
Test accuracy: 89.2%
